In [6]:
from backends import PostgresBackend
import pandas as pd
import psycopg2
import geopandas as gpd
from shapely import wkb
from shapely.geometry import Point
import numpy as np

In [11]:
def moment_calc(df):
	#Calculate the centroid
	centroid = Point([np.mean(np.array([i.coords.xy[0][0] for i in df['geom']])),
					  np.mean(np.array([i.coords.xy[1][0] for i in df['geom']]))])
	#Return mean distance to centroid (in degrees)
	return np.mean([i.distance(centroid) for i in df['geom']])


def query_method(keys):
	
	query = """
	SELECT """ + ','.join(keys)+ """
	FROM incidentaddress
    limit 1000
	"""

	with PostgresBackend(dict(service='nfirs')) as backend:
	    results = backend.query(query=query).fetchall()

	#storing all the values into a dictionary and then into a pandas dataframe
	d = {}
	for key, value in enumerate(keys):
		d[value] = [results[i][key] for i in range(len(results))]

	df = pd.DataFrame(d)
	df.dropna(inplace=True)
	df['geom'] = df['geom'].apply(lambda x: wkb.loads(x, hex=True))

	return df


#Perform the query
df = query_method(keys=['fdid', 'state', 'geom'])
#Group by departments and calculate average distance (moment)
df = df.groupby(['fdid','state']).apply(lambda x: pd.Series({'mean_dist': moment_calc(x), 'count': len(x)})).reset_index()
df.to_pickle('avg_dist')

In [12]:
df = pd.read_pickle('avg_dist')
df

,fdid,state,count,mean_dist
0,02111,PA,1.0,0.000000
1,02169,PA,1.0,0.000000
2,02231,PA,2.0,0.000011
3,02290,PA,1.0,0.000000
4,03020,PA,1.0,0.000000
5,03072,FL,211.0,0.082388
6,03090,PA,8.0,0.020111
7,03100,PA,16.0,0.033741
8,03110,PA,1.0,0.000000
9,03130,PA,8.0,0.066834


,fdid,state,count,mean_dist
0,03072,FL,26.0,0.068693
1,36011,PA,2.0,0.004907
2,36013,PA,4.0,0.007767
3,36017,PA,1.0,0.000000
4,36018,PA,7.0,0.014437
5,36035,PA,1.0,0.000000
6,36039,PA,11.0,0.020812
7,36042,PA,1.0,0.000000
8,36071,PA,1.0,0.000000
9,36075,PA,4.0,0.020223
